In [1]:
import requests
import requests_cache
requests_cache.install_cache('api_cache')

CLIENT_ID = 'b7ed932d39f34723a9fdb6e36d085e5f'
CLIENT_SECRET = '1b89962c2e954fd5bfb48a9c782c2603'

In [2]:
url = 'https://accounts.spotify.com/api/token'

headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

body = {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET
}

response = requests.post(url, headers=headers, data=body)
if (response.status_code != 200):
    raise Exception("Can't get credentials!")
    
ACCESS_TOKEN = response.json()['access_token']

In [3]:
auth_headers = {
    'Authorization': 'Bearer ' + ACCESS_TOKEN
}

response = requests.get('https://api.spotify.com/v1/artists/2d0hyoQ5ynDBnkvAbJKORj', headers=auth_headers)
response.json()

{'external_urls': {'spotify': 'https://open.spotify.com/artist/2d0hyoQ5ynDBnkvAbJKORj'},
 'followers': {'href': None, 'total': 5467829},
 'genres': ['alternative metal',
  'alternative rock',
  'conscious hip hop',
  'funk metal',
  'hard rock',
  'nu metal',
  'political hip hop',
  'post-grunge',
  'rap metal',
  'rap rock',
  'rock'],
 'href': 'https://api.spotify.com/v1/artists/2d0hyoQ5ynDBnkvAbJKORj',
 'id': '2d0hyoQ5ynDBnkvAbJKORj',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/ab6761610000e5ebda4bd2b213cae330e2a4a901',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/ab67616100005174da4bd2b213cae330e2a4a901',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/ab6761610000f178da4bd2b213cae330e2a4a901',
   'width': 160}],
 'name': 'Rage Against The Machine',
 'popularity': 71,
 'type': 'artist',
 'uri': 'spotify:artist:2d0hyoQ5ynDBnkvAbJKORj'}

In [4]:
response.json()['popularity']

71

In [5]:
response = requests.get('https://api.spotify.com/v1/tracks/2pnzeNwo27LdzSnTIWLtEs', headers=auth_headers)
response.status_code

200

In [6]:
response.json()['album']['release_date']

'2023-06-10'

# Searching For Songs

In [7]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [8]:
def add_track(track_info, df):

    track_id = track_info['id']
    release_date = track_info['album']['release_date']
    name = track_info['name']
    artists = [artist_info['name'] for artist_info in track_info['artists']]
    artist_ids = [artist_info['id'] for artist_info in track_info['artists']]
    explicit = track_info['explicit']
    popularity = track_info['popularity']

    row = [track_id, release_date, name, artists, artist_ids, explicit, popularity]

    df.loc[len(df)] = row

In [9]:
def add_tracks(tracks, df):
    for track in tracks:
        add_track(track, df)

In [10]:
def search(q, type, limit, offset):
    body = {
        'q': q,
        'type': type,
        'limit': limit,
        'offset': offset
    }

    response = requests.get('https://api.spotify.com/v1/search', headers=auth_headers, params=body)
    
    if response.status_code != 200:
        raise Exception(f"Invalid response!\n{response.url}\n{response.text}")
        
    return response.json()

In [11]:
def search_tracks(q, limit=50, offset=0):
    body = {
        'q': q,
        'type': 'track',
        'limit': limit,
        'offset': offset
    }

    response = requests.get('https://api.spotify.com/v1/search', headers=auth_headers, params=body)
    
    print(offset, "-", int(offset)+int(limit))
    print("-> target url:", response.url)
    
    if response.status_code != 200:
        raise Exception(f"Invalid response!\n{response.url}\n{response.text}")
        
    return response.json()['tracks']

In [17]:
def find_songs(year):
    songs = pd.DataFrame(columns=['id', 'release_date', 'name', 'artists', 'artist_ids', 'explicit', 'popularity'])
    
    try:
        data = search_tracks(f'year:{year}')
        add_tracks(data['items'], songs)

        while data['next'] is not None:
            next_url = data['next']

            offset, limit = [param.split('=')[-1] for param in next_url.split('&')[-2:]]
            print("-> tnext taret:", next_url)

            data = search_tracks(f'year:{year}', limit=limit, offset=offset)
            add_tracks(data['items'], songs)
    except:
        pass
    
    print('-' * 16)
    print("Got", len(songs), "songs.")
        
    return songs

In [19]:
# songs = find_songs(2023)

# Songs from Billboard Artists

In [20]:
BASE_COLUMNS = ['id', 'release_date', 'name', 'artists', 'artist_ids', 'explicit', 'popularity']

In [21]:
artists = [
    
    # Billboard charts 2022
    
    # top 10
    'Bad Bunny', 'Taylor Swift', 'Harry Styles', 'Drake', 'Morgan Wallen', 
    'Doja Cat', 'Ed Sheeran', 'Adele', 'The weeknd', 'Lil Baby', 
    # top 20
    'Future', 'Justin Bieber', 'Post Malone', 'Jack Harlow', 'Kendrick Lamar', 
    'Luke Combs', 'Juice WRLD', 'Glass Animals', 'Lil Durk', 'Lil Nas X',
    # top 30
    'Dua Lipa', 'Elton John', 'YoungBoy Never Broke Again', 'Rod Wave', 'Kanye West', 
    'Olivia Rodrigo', 'Beyoncé', 'The Kid LAROI', 'Billie Eilish', 'Kodak Black',
    # top 40
    'Lizzo', 'Gunna', 'Imagine Dragons', 'Latto', 'Chris Stapleton', 
    'Summer Walker', 'Walker Hayes', 'Polo G', 'SZA', 'Bruno Mars',
    # top 50
    'Zach Bryan', 'Steve Lacy', 'Eminem', 'Bailey Zimmerman', 'Tyler, The Creator', 
    'Kate Bush', 'Karol G', 'Kane Brown', 'GAYLE', 'Nardo Wick',
    
    # some time popular
    'Cardi B', 'DaBaby', 'BTS', 'Panic! At The Disco', 'Jonas Brother', 'Halsey', 
    'Travis Scott', 'Khalid', 'Ariana Grande', 'Shawn Mendes', 'Queen', 'XXXTENTACION', 
    '21 Savage', 'Marshmello', 'Maroon 5', 'The 1975', 'Lil Peep', 'P!nk', 'G-Eazy', 'NF',
    
    # special
    'Nirvana', 'Led Zeppelin', 'Aerosmith', 'The Beatles', 'Pixies',
    
    # russian artists
    'Nikitata', 'Morgenshtern', 'Skryptonite ', 'kizaru', 'LSP', 'Pharaoh', 
    'Boulevard Depo', 'Slava Marlow', 'GONE.Fludd', 'Mayot', 'Макс Корж', 
    'Artik & Asti', 'HammAli & Navai', 'GAYAZOV$ BROTHER$', 'Korol i Shut', 
    'MiyaGi & Эндшпиль', 
    'Дора', 'нексюша',
    
    # not popular artists
    'American Footbal', 'Blur', 'Yellowcard', 'Basement', 'Pacifica', 'Jet', 'Sex Pistols', 
    'Radiohead', 'The Vines', 'Kurt Cobain', 'конец солнечных дней', 'Neverlove', 'LIZER', 
    'Джизус', 'курящих нет', 'DSPRITE', 'LEANRODE', 'LXNER', 'Violent Soho', 'Soundgarden', 
    'Polyphia', 'Stone Temple Pilots', 'The Jins', 'Три дня дождя', 'Misfits', 'Дайте танк (!)', 
    'LOVV66', 'grandson', 'Cigarettes After Sex', 'тринадцать карат', 'Hozier', 'Dominic Miller', 
    'IVOXYGEN', 'WILLOW', 'Linkin Park', 'Three Days Grace', 'RAMMSTEIN', 'Coldplay', 'ANIKV', 
    'Skrillex', 'Земфира', 'REDZED', 'cold carti', 'blink-182', 'System of a Down', 'Deftones', 
    'KxllSwxtch', 'КОСМОНАВТОВ НЕТ', 'Nickelback', 'madk1d', 'FEDUK', 'Кино', 'Green Day'
]

len(artists)

146

In [22]:
def get_artist_songs(artist_name, first_n_albums=5):
    songs = pd.DataFrame(columns=BASE_COLUMNS)
    
    artist = search(artist_name, 'artist', 50, 0)['artists']['items'][0]
    
    albums = search(f'artist:{artist_name}', 'album', 50, 0)['albums']['items']
    
    for i, album in enumerate(albums[:first_n_albums]):
        
        print(f"{i+1}/{first_n_albums} albums")
        response = requests.get(f"https://api.spotify.com/v1/albums/{album['id']}/tracks", headers=auth_headers)
        if response.status_code != 200:
            raise Excpetion("Invalid response!")
            
        album_songs = response.json()['items']
        album_length = len(album_songs)
        for j, song in enumerate(album_songs):
#             print(f"  {j+1}/{album_length} songs")
            response = requests.get(f"https://api.spotify.com/v1/tracks/{song['id']}", headers=auth_headers)
            if response.status_code != 200:
                raise Excpetion("Invalid response!")
                
            add_track(response.json(), songs)
    
    return songs

In [45]:
songs_from_artists = pd.DataFrame(columns=BASE_COLUMNS)

for artist in artists:
    try:
        print(f'[{artist}]')
        songs_from_artists = pd.concat([songs_from_artists, get_artist_songs(artist)])
    except:
        print("Can't get", artist)
        continue

[Bad Bunny]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Taylor Swift]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Harry Styles]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Drake]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Morgan Wallen]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Doja Cat]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Ed Sheeran]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Adele]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[The weeknd]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Lil Baby]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Future]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Justin Bieber]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Post Malone]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Jack Harlow]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Kendrick Lamar]
1/5 albums
2/5 albums
3/5 albums
4/5 a

5/5 albums
[Dominic Miller]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[IVOXYGEN]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[WILLOW]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Linkin Park]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Three Days Grace]
Can't get Three Days Grace
[RAMMSTEIN]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Coldplay]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[ANIKV]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Skrillex]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Земфира]
1/5 albums
2/5 albums
[REDZED]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[cold carti]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[blink-182]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[System of a Down]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[Deftones]
Can't get Deftones
[KxllSwxtch]
1/5 albums
2/5 albums
3/5 albums
4/5 albums
5/5 albums
[КОСМОН

In [46]:
songs_from_artists

,id,release_date,name,artists,artist_ids,explicit,popularity
0,6Xom58OOXk2SoU711L2IXO,2022-05-06,Moscow Mule,[Bad Bunny],[4q3ewBCX7sLwd24euuV69X],True,88
1,1dm6z1fWB0cErMszU25dy2,2022-05-06,Después de la Playa,[Bad Bunny],[4q3ewBCX7sLwd24euuV69X],False,82
2,6Sq7ltF9Qa7SNFBsV5Cogx,2022-05-06,Me Porto Bonito,"[Bad Bunny, Chencho Corleone]","[4q3ewBCX7sLwd24euuV69X, 37230BxxYs9ksS7OkZw3IU]",True,92
3,1IHWl5LamUGEuP4ozKQSXZ,2022-05-06,Tití Me Preguntó,[Bad Bunny],[4q3ewBCX7sLwd24euuV69X],True,90
4,5CzixCxDkRXX9mScCmah8O,2022-05-06,Un Ratito,[Bad Bunny],[4q3ewBCX7sLwd24euuV69X],True,81
...,...,...,...,...,...,...,...
71,1nLnpLXvl68RZCSjfkyiaa,1995-10-10,Brain Stew,[Green Day],[7oPftvlwr6VrsViSDV7fJY],True,73
72,6uWp8yAt8dN5ZaT7REJ6RV,1995-10-10,Jaded,[Green Day],[7oPftvlwr6VrsViSDV7fJY],True,54
73,40u3oEy2Zh04fh67QNBJlv,1995-10-10,Westbound Sign,[Green Day],[7oPftvlwr6VrsViSDV7fJY],False,43
74,4zz1q7N4PJGYePSb39wUZF,1995-10-10,Tight Wad Hill,[Green Day],[7oPftvlwr6VrsViSDV7fJY],False,40


# Artist Info

In [47]:
song_followers = []
song_popularity = []

for i, row in songs_from_artists.iterrows():
    
    artist_ids = row['artist_ids']
    body = {
        'ids': ",".join(artist_ids)
    }
    response = requests.get('https://api.spotify.com/v1/artists', headers=auth_headers, params=body)
    if response.status_code != 200:
        print("Can't get artist info!")
        break
        
    song_artists_info = []
    for artist in response.json()['artists']:
        followers = artist['followers']['total']
        popularity = artist['popularity']
        song_artists_info.append((followers, popularity))
        
    if len(response.json()['artists']) > 1:
        song_followers.append(sum([followers * popularity / 100 for followers, popularity in song_artists_info]))
        song_popularity.append(sum([popularity for followers, popularity in song_artists_info]) / len(song_artists_info))
    else:
        song_followers.append(song_artists_info[0][0])
        song_popularity.append(song_artists_info[0][1])
    
songs_from_artists['artists_followers_combined'] = song_followers
songs_from_artists['aritsts_mean_popularity'] = song_popularity

In [48]:
songs_from_artists = songs_from_artists[['id', 'release_date', 'name', 'artists', 'artist_ids', 'aritsts_mean_popularity', 'artists_followers_combined', 'explicit', 'popularity']]

In [49]:
songs_from_artists['explicit'] = songs_from_artists['explicit'].astype(int)

In [50]:
songs_from_artists = songs_from_artists.reset_index(drop=True)
songs_from_artists

,id,release_date,name,artists,artist_ids,aritsts_mean_popularity,artists_followers_combined,explicit,popularity
0,6Xom58OOXk2SoU711L2IXO,2022-05-06,Moscow Mule,[Bad Bunny],[4q3ewBCX7sLwd24euuV69X],96.0,70146841.00,1,88
1,1dm6z1fWB0cErMszU25dy2,2022-05-06,Después de la Playa,[Bad Bunny],[4q3ewBCX7sLwd24euuV69X],96.0,70146841.00,0,82
2,6Sq7ltF9Qa7SNFBsV5Cogx,2022-05-06,Me Porto Bonito,"[Bad Bunny, Chencho Corleone]","[4q3ewBCX7sLwd24euuV69X, 37230BxxYs9ksS7OkZw3IU]",88.0,67728144.16,1,92
3,1IHWl5LamUGEuP4ozKQSXZ,2022-05-06,Tití Me Preguntó,[Bad Bunny],[4q3ewBCX7sLwd24euuV69X],96.0,70146841.00,1,90
4,5CzixCxDkRXX9mScCmah8O,2022-05-06,Un Ratito,[Bad Bunny],[4q3ewBCX7sLwd24euuV69X],96.0,70146841.00,1,81
...,...,...,...,...,...,...,...,...,...
7535,1nLnpLXvl68RZCSjfkyiaa,1995-10-10,Brain Stew,[Green Day],[7oPftvlwr6VrsViSDV7fJY],79.0,14076517.00,1,73
7536,6uWp8yAt8dN5ZaT7REJ6RV,1995-10-10,Jaded,[Green Day],[7oPftvlwr6VrsViSDV7fJY],79.0,14076517.00,1,54
7537,40u3oEy2Zh04fh67QNBJlv,1995-10-10,Westbound Sign,[Green Day],[7oPftvlwr6VrsViSDV7fJY],79.0,14076517.00,0,43
7538,4zz1q7N4PJGYePSb39wUZF,1995-10-10,Tight Wad Hill,[Green Day],[7oPftvlwr6VrsViSDV7fJY],79.0,14076517.00,0,40


# Audio Features

In [51]:
features = []

for i in range(0, len(songs_from_artists), 100):
    
    body = {
        'ids': ",".join(list(songs_from_artists['id'].loc[i:i+99]))  # maximum 100
    }

    response = requests.get('https://api.spotify.com/v1/audio-features', headers=auth_headers, params=body)
    
    if response.status_code != 200:
        print(f"Invalid response! ({body})")
        print(f"{response.text}")
        break
        
    print("Appending", len(response.json()['audio_features']), "values")
#     print(response.json()['audio_features'])
    
    features.extend(response.json()['audio_features'])

print("--------------------")
print("Total of", len(features))

Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100 values
Appending 100

In [52]:
len(features)

7540

In [53]:
features = [feature for feature in features if feature is not None]

In [54]:
len(features)

7533

In [55]:
not_to_include = ['type', 'uri', 'track_href', 'analysis_url']

In [56]:
features = [{f: v for f, v in feature.items() if f not in not_to_include} for feature in features]

In [57]:
features_df = pd.DataFrame(features, columns=[column for column in features[0].keys() if column not in not_to_include])

In [58]:
features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,0.804,0.674,5,-5.453,0,0.0333,0.294000,0.000001,0.1150,0.292,99.968,6Xom58OOXk2SoU711L2IXO,245940,4
1,0.564,0.903,5,-3.221,1,0.3070,0.356000,0.000000,0.1750,0.607,78.293,1dm6z1fWB0cErMszU25dy2,230400,4
2,0.911,0.712,1,-5.105,0,0.0817,0.090100,0.000027,0.0933,0.425,92.005,6Sq7ltF9Qa7SNFBsV5Cogx,178567,4
3,0.650,0.715,5,-5.198,0,0.2530,0.099300,0.000291,0.1260,0.187,106.672,1IHWl5LamUGEuP4ozKQSXZ,243717,4
4,0.787,0.546,0,-7.094,0,0.0532,0.305000,0.000046,0.1220,0.222,93.050,5CzixCxDkRXX9mScCmah8O,176936,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7528,0.559,0.543,1,-5.429,1,0.3020,0.001330,0.000020,0.0577,0.337,75.958,1nLnpLXvl68RZCSjfkyiaa,193000,4
7529,0.254,0.995,1,-3.068,1,0.0669,0.000030,0.000041,0.0392,0.843,159.336,6uWp8yAt8dN5ZaT7REJ6RV,90227,4
7530,0.171,0.994,10,-4.121,1,0.0847,0.000011,0.000373,0.1550,0.553,185.403,40u3oEy2Zh04fh67QNBJlv,133267,4
7531,0.159,0.964,1,-5.686,1,0.0479,0.000011,0.001480,0.0743,0.960,197.632,4zz1q7N4PJGYePSb39wUZF,121027,4


# Merging & Saving

In [59]:
song_data_full = pd.merge(songs_from_artists, features_df, on='id', how='left')

In [60]:
song_data_full

,id,release_date,name,artists,artist_ids,aritsts_mean_popularity,artists_followers_combined,explicit,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,6Xom58OOXk2SoU711L2IXO,2022-05-06,Moscow Mule,[Bad Bunny],[4q3ewBCX7sLwd24euuV69X],96.0,70146841.00,1,88,0.804,0.674,5.0,-5.453,0.0,0.0333,0.294000,0.000001,0.1150,0.292,99.968,245940.0,4.0
1,1dm6z1fWB0cErMszU25dy2,2022-05-06,Después de la Playa,[Bad Bunny],[4q3ewBCX7sLwd24euuV69X],96.0,70146841.00,0,82,0.564,0.903,5.0,-3.221,1.0,0.3070,0.356000,0.000000,0.1750,0.607,78.293,230400.0,4.0
2,6Sq7ltF9Qa7SNFBsV5Cogx,2022-05-06,Me Porto Bonito,"[Bad Bunny, Chencho Corleone]","[4q3ewBCX7sLwd24euuV69X, 37230BxxYs9ksS7OkZw3IU]",88.0,67728144.16,1,92,0.911,0.712,1.0,-5.105,0.0,0.0817,0.090100,0.000027,0.0933,0.425,92.005,178567.0,4.0
3,1IHWl5LamUGEuP4ozKQSXZ,2022-05-06,Tití Me Preguntó,[Bad Bunny],[4q3ewBCX7sLwd24euuV69X],96.0,70146841.00,1,90,0.650,0.715,5.0,-5.198,0.0,0.2530,0.099300,0.000291,0.1260,0.187,106.672,243717.0,4.0
4,5CzixCxDkRXX9mScCmah8O,2022-05-06,Un Ratito,[Bad Bunny],[4q3ewBCX7sLwd24euuV69X],96.0,70146841.00,1,81,0.787,0.546,0.0,-7.094,0.0,0.0532,0.305000,0.000046,0.1220,0.222,93.050,176936.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7535,1nLnpLXvl68RZCSjfkyiaa,1995-10-10,Brain Stew,[Green Day],[7oPftvlwr6VrsViSDV7fJY],79.0,14076517.00,1,73,0.559,0.543,1.0,-5.429,1.0,0.3020,0.001330,0.000020,0.0577,0.337,75.958,193000.0,4.0
7536,6uWp8yAt8dN5ZaT7REJ6RV,1995-10-10,Jaded,[Green Day],[7oPftvlwr6VrsViSDV7fJY],79.0,14076517.00,1,54,0.254,0.995,1.0,-3.068,1.0,0.0669,0.000030,0.000041,0.0392,0.843,159.336,90227.0,4.0
7537,40u3oEy2Zh04fh67QNBJlv,1995-10-10,Westbound Sign,[Green Day],[7oPftvlwr6VrsViSDV7fJY],79.0,14076517.00,0,43,0.171,0.994,10.0,-4.121,1.0,0.0847,0.000011,0.000373,0.1550,0.553,185.403,133267.0,4.0
7538,4zz1q7N4PJGYePSb39wUZF,1995-10-10,Tight Wad Hill,[Green Day],[7oPftvlwr6VrsViSDV7fJY],79.0,14076517.00,0,40,0.159,0.964,1.0,-5.686,1.0,0.0479,0.000011,0.001480,0.0743,0.960,197.632,121027.0,4.0


In [62]:
song_data_full = song_data_full.drop_duplicates('id')
song_data_full = song_data_full.dropna().reset_index(drop=True)

In [63]:
song_data_full['key'] = song_data_full['key'].astype(int, errors='ignore')
song_data_full['time_signature'] = song_data_full['time_signature'].astype(int, errors='ignore')
song_data_full['duration_ms'] = song_data_full['duration_ms'].astype(int, errors='ignore')
song_data_full['mode'] = song_data_full['mode'].astype(int, errors='ignore')

In [64]:
song_data_full

,id,release_date,name,artists,artist_ids,aritsts_mean_popularity,artists_followers_combined,explicit,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,6Xom58OOXk2SoU711L2IXO,2022-05-06,Moscow Mule,[Bad Bunny],[4q3ewBCX7sLwd24euuV69X],96.0,70146841.00,1,88,0.804,0.674,5,-5.453,0,0.0333,0.294000,0.000001,0.1150,0.292,99.968,245940,4
1,1dm6z1fWB0cErMszU25dy2,2022-05-06,Después de la Playa,[Bad Bunny],[4q3ewBCX7sLwd24euuV69X],96.0,70146841.00,0,82,0.564,0.903,5,-3.221,1,0.3070,0.356000,0.000000,0.1750,0.607,78.293,230400,4
2,6Sq7ltF9Qa7SNFBsV5Cogx,2022-05-06,Me Porto Bonito,"[Bad Bunny, Chencho Corleone]","[4q3ewBCX7sLwd24euuV69X, 37230BxxYs9ksS7OkZw3IU]",88.0,67728144.16,1,92,0.911,0.712,1,-5.105,0,0.0817,0.090100,0.000027,0.0933,0.425,92.005,178567,4
3,1IHWl5LamUGEuP4ozKQSXZ,2022-05-06,Tití Me Preguntó,[Bad Bunny],[4q3ewBCX7sLwd24euuV69X],96.0,70146841.00,1,90,0.650,0.715,5,-5.198,0,0.2530,0.099300,0.000291,0.1260,0.187,106.672,243717,4
4,5CzixCxDkRXX9mScCmah8O,2022-05-06,Un Ratito,[Bad Bunny],[4q3ewBCX7sLwd24euuV69X],96.0,70146841.00,1,81,0.787,0.546,0,-7.094,0,0.0532,0.305000,0.000046,0.1220,0.222,93.050,176936,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7528,1nLnpLXvl68RZCSjfkyiaa,1995-10-10,Brain Stew,[Green Day],[7oPftvlwr6VrsViSDV7fJY],79.0,14076517.00,1,73,0.559,0.543,1,-5.429,1,0.3020,0.001330,0.000020,0.0577,0.337,75.958,193000,4
7529,6uWp8yAt8dN5ZaT7REJ6RV,1995-10-10,Jaded,[Green Day],[7oPftvlwr6VrsViSDV7fJY],79.0,14076517.00,1,54,0.254,0.995,1,-3.068,1,0.0669,0.000030,0.000041,0.0392,0.843,159.336,90227,4
7530,40u3oEy2Zh04fh67QNBJlv,1995-10-10,Westbound Sign,[Green Day],[7oPftvlwr6VrsViSDV7fJY],79.0,14076517.00,0,43,0.171,0.994,10,-4.121,1,0.0847,0.000011,0.000373,0.1550,0.553,185.403,133267,4
7531,4zz1q7N4PJGYePSb39wUZF,1995-10-10,Tight Wad Hill,[Green Day],[7oPftvlwr6VrsViSDV7fJY],79.0,14076517.00,0,40,0.159,0.964,1,-5.686,1,0.0479,0.000011,0.001480,0.0743,0.960,197.632,121027,4


In [66]:
song_data_full.to_csv('data/my_dataset_from_artists.csv', index=False)